In [1]:
import pandas as pd
import numpy as np
import copy
import os
from tqdm import tqdm

In [2]:
import matplotlib.pyplot as plt

In [3]:
benchmark_directory = "/projects/mcmm/results_final/"

In [4]:
def get_info_from_filename(np_fname):
    splitted = np_fname.lower().split("_")
    func = splitted[0]
    dim = int(splitted[1])
    
    algorithm = None
    if "basinhopping" in np_fname:
        assert algorithm is None; algorithm = "basinhopping"
    elif "differential_evolution" in np_fname:
        assert algorithm is None; algorithm = "differential_evolution"
    elif "direct" in np_fname:
        assert algorithm is None; algorithm = "direct"
    elif "dual_annealing" in np_fname:
        assert algorithm is None; algorithm = "dual_annealing"
    elif "shgo" in np_fname:
        assert algorithm is None; algorithm = "shgo"
    elif "simulated_annealing" in np_fname:
        assert algorithm is None; algorithm = "simulated_annealing"
    elif "mcir" in np_fname:
        assert algorithm is None; algorithm = "mcir"
    
    assert algorithm is not None, "algorithm not found in {}".format(np_fname)
    
    info = {
        "function": func,
        "dimension": dim,
        "algorithm": algorithm,
    }
    return info

In [5]:
all_results = []

for filename in os.listdir(benchmark_directory):
    assert filename.endswith("_Y.npy")
    result_dict = get_info_from_filename(filename)
    result_dict['obj_list'] = np.load(os.path.join(benchmark_directory, filename))
    all_results.append(result_dict)

In [6]:
# Generate dataframe for results

In [7]:
def process_list(nums):
    result = []
    min_num = float('inf')  # initialize minimum value to positive infinity
    for num in nums:
        if num < min_num:
            min_num = num
        result.append(min_num)
    return result

In [8]:
result_df = pd.DataFrame(all_results)
result_df['min_found_list'] = result_df['obj_list'].apply(process_list)
result_df = result_df.sort_values(["function", "dimension", "algorithm"])
result_df.to_pickle("result_df.pkl")
result_df

,function,dimension,algorithm,obj_list,min_found_list
53,ackley,10,basinhopping,"[21.23412, 21.23412, 21.23412, 21.23412, 21.23...","[21.23412, 21.23412, 21.23412, 21.23412, 21.23..."
90,ackley,10,basinhopping,"[21.551155, 21.551155, 21.551155, 21.551155, 2...","[21.551155, 21.551155, 21.551155, 21.551155, 2..."
285,ackley,10,basinhopping,"[21.037682, 21.037682, 21.037682, 21.037682, 2...","[21.037682, 21.037682, 21.037682, 21.037682, 2..."
296,ackley,10,basinhopping,"[20.554607, 20.554607, 20.554607, 20.554607, 2...","[20.554607, 20.554607, 20.554607, 20.554607, 2..."
365,ackley,10,basinhopping,"[20.38853, 20.38853, 20.38853, 20.38853, 20.38...","[20.38853, 20.38853, 20.38853, 20.38853, 20.38..."
...,...,...,...,...,...
49,watson,32,shgo,[],[]
124,watson,32,shgo,[],[]
183,watson,32,shgo,[],[]
241,watson,32,shgo,[],[]


In [13]:
# Generate statistics for results

In [14]:
stat_df = result_df.groupby(['function', 'dimension', 'algorithm']).agg(
    obj_lists=('obj_list', list),
    min_found_lists=('min_found_list', list)
).reset_index()
stat_df

,function,dimension,algorithm,obj_lists,min_found_lists
0,ackley,10,basinhopping,"[[21.23412, 21.23412, 21.23412, 21.23412, 21.2...","[[21.23412, 21.23412, 21.23412, 21.23412, 21.2..."
1,ackley,10,differential_evolution,"[[20.970161, 21.383083, 21.541191, 21.367462, ...","[[20.970161, 20.970161, 20.970161, 20.970161, ..."
2,ackley,10,direct,"[[4.552539, 17.018728, 17.45362, 17.018728, 17...","[[4.552539, 4.552539, 4.552539, 4.552539, 4.55..."
3,ackley,10,dual_annealing,"[[21.551155, 21.254986, 21.6148, 20.65361, 21....","[[21.551155, 21.254986, 21.254986, 20.65361, 2..."
4,ackley,10,mcir,"[[21.122943878173828, 21.122943878173828, 21.1...","[[21.122943878173828, 21.122943878173828, 21.1..."
...,...,...,...,...,...
97,watson,32,differential_evolution,"[[307303.9192011102, 300601.11886811716, 20247...","[[307303.9192011102, 300601.11886811716, 30060..."
98,watson,32,direct,"[[528043.0017411856, 528043.0017411856, 528043...","[[528043.0017411856, 528043.0017411856, 528043..."
99,watson,32,dual_annealing,"[[452408.46655145474, 37596435.013013594, 6810...","[[452408.46655145474, 452408.46655145474, 6810..."
100,watson,32,mcir,"[[651574.6720559866, 651574.6720559866, 651574...","[[651574.6720559866, 651574.6720559866, 651574..."


In [15]:
def calculate_mean_and_std_of_lists(lists):
    pd_lists = pd.DataFrame(lists)
    means = pd_lists.mean(axis = 0).to_numpy()
    std = pd_lists.std(axis=0).fillna(0).to_numpy()
    
    return {
        "avg_min_found": means,
        "std_min_found": std
    }

In [16]:
applied_df = stat_df.apply(
    lambda row: calculate_mean_and_std_of_lists(row.min_found_lists), 
    axis='columns', 
    result_type='expand'
)
stat_df = pd.concat([stat_df, applied_df], axis='columns')

In [17]:
stat_df.to_pickle("result_df.pkl")
stat_df

,function,dimension,algorithm,obj_lists,min_found_lists,avg_min_found,std_min_found
0,ackley,10,basinhopping,"[[21.23412, 21.23412, 21.23412, 21.23412, 21.2...","[[21.23412, 21.23412, 21.23412, 21.23412, 21.2...","[20.95322036743164, 20.95322036743164, 20.9532...","[0.4799298346042633, 0.4799298346042633, 0.479..."
1,ackley,10,differential_evolution,"[[20.970161, 21.383083, 21.541191, 21.367462, ...","[[20.970161, 20.970161, 20.970161, 20.970161, ...","[21.358928680419922, 21.04205322265625, 20.861...","[0.23468589782714844, 0.45797276496887207, 0.3..."
2,ackley,10,direct,"[[4.552539, 17.018728, 17.45362, 17.018728, 17...","[[4.552539, 4.552539, 4.552539, 4.552539, 4.55...","[4.552539, 4.552539, 4.552539, 4.552539, 4.552...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,ackley,10,dual_annealing,"[[21.551155, 21.254986, 21.6148, 20.65361, 21....","[[21.551155, 21.254986, 21.254986, 20.65361, 2...","[20.95322036743164, 20.893985748291016, 20.893...","[0.4799298346042633, 0.39916253089904785, 0.39..."
4,ackley,10,mcir,"[[21.122943878173828, 21.122943878173828, 21.1...","[[21.122943878173828, 21.122943878173828, 21.1...","[20.95499038696289, 20.95499038696289, 20.9549...","[0.4690623126870676, 0.4690623126870676, 0.469..."
...,...,...,...,...,...,...,...
97,watson,32,differential_evolution,"[[307303.9192011102, 300601.11886811716, 20247...","[[307303.9192011102, 300601.11886811716, 30060...","[6844500.333347577, 1662800.6946316964, 136326...","[11484157.537096405, 1726414.08927726, 1784276..."
98,watson,32,direct,"[[528043.0017411856, 528043.0017411856, 528043...","[[528043.0017411856, 528043.0017411856, 528043...","[528043.0017411856, 528043.0017411856, 528043....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
99,watson,32,dual_annealing,"[[452408.46655145474, 37596435.013013594, 6810...","[[452408.46655145474, 452408.46655145474, 6810...","[37751606.73584604, 3791722.58588769, 2881944....","[62101331.395239055, 5219180.813468672, 538362..."
100,watson,32,mcir,"[[651574.6720559866, 651574.6720559866, 651574...","[[651574.6720559866, 651574.6720559866, 651574...","[588816.5247719139, 588816.5247719139, 588816....","[88753.42303854393, 88753.42303854393, 88753.4..."
